In [4]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from glob import glob

# --- CONFIG ---
# UPDATE THIS PATH if your input path is different (Check the "Data" tab on the right)
# Based on your screenshot, it's likely one of these:
input_data_dir = '/kaggle/input/inbreast-png-dataset/Inbreast_png' 
excel_path = '/kaggle/input/inbreast-png-dataset/INbreast.xls'

# Where we will build the new dataset
output_dir = '/kaggle/working/INbreast_Formatted'

# --- 1. Load Labels ---
print("Loading Excel...")
try:
    df = pd.read_excel(excel_path, sheet_name=0) # Sheet1
    # Standardize column names (strip spaces, specific to INbreast messiness)
    df.columns = [c.strip() for c in df.columns]
    
    # Check for the right columns. Usually 'File Name' and 'Bi-Rads'
    print(f"Columns found: {list(df.columns)}")
    
    if 'File Name' not in df.columns or 'Bi-Rads' not in df.columns:
        raise ValueError("Could not find 'File Name' or 'Bi-Rads' columns!")
        
except Exception as e:
    print(f"Error reading Excel: {e}")
    # Fallback: Sometimes paths are slightly different in Kaggle
    print("Please check the exact path of your .xls file in the sidebar.")

Loading Excel...
Columns found: ['Patient ID', 'Patient age', 'Laterality', 'View', 'Acquisition date', 'File Name', 'ACR', 'Bi-Rads', 'Mass', 'Micros', 'Distortion', 'Asymmetry', 'Findings Notes (in Portuguese)', 'Other Notes', 'Lesion Annotation Status', 'Pectoral Muscle Annotation', 'Other Annotations']


In [5]:

# --- 2. Setup Folders ---
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)


In [11]:

# --- 3. Match Images to Labels ---
valid_entries = [] # Store (image_path, label)
missing_count = 0

print("Scanning images...")
for index, row in df.iterrows():
    # --- SAFETY CHECK: Skip empty rows ---
    if pd.isna(row['File Name']):
        continue
        
    # --- FIX: Convert to int safely ---
    try:
        file_id = str(int(row['File Name'])) 
    except ValueError:
        # If it's a string like "22678622_new", just use it as is
        file_id = str(row['File Name'])

    # Get Label
    if pd.isna(row['Bi-Rads']):
        continue # Skip if label is missing too
        
    label = str(row['Bi-Rads']).replace(' ', '_') 
    
    # Simplify 4a/4b/4c -> 4
    if label.startswith('4'):
        label = '4'
    
    # Construct path
    src_path = os.path.join(input_data_dir, file_id + '.png')
    
    if os.path.exists(src_path):
        valid_entries.append((src_path, label))
    else:
        # Optional: Print only the first few missing files to avoid clutter
        if missing_count < 3: 
            print(f"Failed to find: {src_path}")
        missing_count += 1

print(f"Found {len(valid_entries)} valid images. Missing/Mismatch: {missing_count}")


Scanning images...
Found 410 valid images. Missing/Mismatch: 0


In [12]:

# --- 4. Split and Copy ---
train_data, val_data = train_test_split(valid_entries, test_size=0.15, random_state=42, stratify=[x[1] for x in valid_entries])

def copy_files(data_list, split_name):
    print(f"Processing {split_name}...")
    for src, label in data_list:
        # Create class folder if not exists (e.g., train/4)
        class_dir = os.path.join(output_dir, split_name, f"Bi-Rads_{label}")
        os.makedirs(class_dir, exist_ok=True)
        
        # Copy
        dst = os.path.join(class_dir, os.path.basename(src))
        shutil.copy(src, dst)

copy_files(train_data, 'train')
copy_files(val_data, 'val')

print(f"\nSUCCESS! Dataset ready at: {output_dir}")

Processing train...
Processing val...

SUCCESS! Dataset ready at: /kaggle/working/INbreast_Formatted


In [14]:
# 1. Uninstall ALL conflicting libraries
!pip uninstall -y numpy opencv-python opencv-python-headless ultralytics roboflow

# 2. Reinstall them in a strict order (Headless OpenCV is better for Kaggle)
!pip install "numpy<2.0" "opencv-python-headless<4.10" ultralytics roboflow

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: opencv-python 4.12.0.88
Uninstalling opencv-python-4.12.0.88:
  Successfully uninstalled opencv-python-4.12.0.88
Found existing installation: opencv-python-headless 4.12.0.88
Uninstalling opencv-python-headless-4.12.0.88:
  Successfully uninstalled opencv-python-headless-4.12.0.88
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of roboflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of roboflow to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you wa

In [ ]:
from ultralytics import YOLO

# 1. Load the Nano model
model = YOLO('yolov8n-cls.pt') 

# 2. Train with "On-the-Fly" Augmentation
results = model.train(
    data='/kaggle/working/INbreast_Formatted', 
    epochs=50, 
    imgsz=224, 
    batch=16, 
    name='inbreast_aug_model',
    amp=False,
    
    # --- AUGMENTATION HYPERPARAMETERS ---
    degrees=25.0,      # Rotate images +/- 25 degrees (Simulates patient positioning)
    fliplr=0.5,        # 50% chance to flip Left/Right (Simulates Left vs Right breast)
    scale=0.3,         # Zoom image +/- 30% (Simulates different breast sizes)
    shear=5.0,         # Slight shear distortion
    erasing=0.2,       # Randomly erase small box (Helps robustness against artifacts)
    
    # --- MEDICAL SAFETY (Turn off color changes) ---
    hsv_h=0.0,         # Don't change Hue (X-rays don't have hue)
    hsv_s=0.0,         # Don't change Saturation
    hsv_v=0.1          # Slight Brightness change is okay (Exposure diffs)
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.236 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/INbreast_Formatted, degrees=25.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.2, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.1, imgsz=224, int8=False, iou=0.7,